# Automatický zpětný průchod v Pytorch: autograd

- Knihovna Pytorch umožňuje automatický výpočet gradientů (autograd) a a jejich zpětnou propagaci.
- Pokud vytvoříme jakýkoliv `Tensor` s dodatečným parametrem `requires_grad=True`, knihovna si zapamatuje všechny operace s ním provedené.
- Zkusme nejprve velmi jednoduchý příklad násobení dvou tensorů s `requires_grad=True`.

In [1]:
import torch

In [2]:
u = torch.tensor(2., requires_grad=True)
u

tensor(2., requires_grad=True)

In [3]:
v = torch.tensor(3., requires_grad=True)
v

tensor(3., requires_grad=True)

In [4]:
w = u * v
w, w.requires_grad

(tensor(6., grad_fn=<MulBackward0>), True)

Výsledkem násobení $2 \cdot 3$ je nepřekvapivě hodnota $6$. Můžeme si ale všimnout dvou věcí:
- Výsledný tensor `w` má rovněž nastaveno `requires_grad=True`.
- U tensoru se objevil atribut `grad_fn=<MulBackward0>`. Ten značí funkci, která se bude volat při zpětném průchodu tak, aby mohl být příchozí gradient propagován grafem dále až ke vstupům, kterými jsou v našem příkladu `u` a `v`.

In [5]:
w.grad_fn

Tím, že si každý tensor skrze atribut `grad_fn` pamatuje, z jaké operace pochází, resp. kterou funkci má volat, pokud se k němu při zpětném průchodu dostane příchozí gradient, vzniká orientovaný výpočetní graf.
- Uzly jsou reprezentované proměnnými (např. tensory `u`, `v` a `w`) či operacemi nad nimi (např. `MulBackward0`).
- Hrany jsou definovány odkazy na své předky - např. `w.grad_fn` odkazuje na "rodiče" (parent) `MulBackward0` uzlu `w`. Tento parent je funkce, která umí převzít příchozí gradient a řetízkovým pravidlem propaguje dál na své vstupy. Své vlastní předky má `MulBackward0` uloženy v atributu `next_functions`.

In [6]:
w.grad_fn.next_functions

((<AccumulateGrad at 0x2a35f907df0>, 0),
 (<AccumulateGrad at 0x2a35f907220>, 0))

Předkem `MulBackward0` jsou dvě `AccumulateGrad`, které značí přičtení (akumulace) gradientu k nějaké proměnné. Jelikož v našem případě je graf velmi jednoduchý, jedná se již o listy grafu `u` a `v`.

Jak uvidíme dále, gradienty se nepřepisují, ale akumulují tak, aby vše odpovídalo pravidlům diferenciálu a matematické analýzy v případech, kdy uzel má více než jednoho potomka.

In [7]:
u_ = w.grad_fn.next_functions[0][0].variable
u_, u_ is u

(tensor(2., requires_grad=True), True)

Celá zpětná propagace tedy spočívá v tom, že se postupně od konce výpočtů, tj. např. uzlu `w`, až na začátek postupně volají `grad_fn` jednotlivých uzlů a akumulují gradienty jejich přímých rodičů. Spustí se metodou `backward()` na uzlu, který si vybereme, obvykle ten na konci grafu - v našem případě `w`.

In [8]:
w.backward()

Nyní proběhla zpětná propagace a derivace, tj. gradienty na všechny uzly, u kterých bylo nastaveno `requires_grad=True`, se uložily do atributů `grad` jednotlivých proměnných.

In [9]:
u.grad, v.grad

(tensor(3.), tensor(2.))

Výsledek dává smysl, protože $w = u \cdot v$ a tedy $dw / du = v = 3$ a $dw / dv = u = 2$.

# Složená funkce: příklad $z = (x_1 + ax_2)^2$

Pro ověření zkusme příklad z přednášky
$$z = (x_1 + ax_2)^2$$
Pokud dosadíme hodnoty $x_1 = 1$, $a = 2$ a $x_2 = 3$, dostaneme $z = (1 + 2 \cdot 3)^2 = 49$.

In [10]:
x1 = torch.tensor(1., requires_grad=True)
x1

tensor(1., requires_grad=True)

In [11]:
a = torch.tensor(2., requires_grad=True)
a

tensor(2., requires_grad=True)

In [12]:
x2 = torch.tensor(3., requires_grad=True)
x2

tensor(3., requires_grad=True)

In [13]:
y = x1 + a * x2
y

tensor(7., grad_fn=<AddBackward0>)

In [14]:
z = y ** 2
z

tensor(49., grad_fn=<PowBackward0>)

PyTorch autograd si nyní pamatuje celou historii výpočtů od `x1`, `a`, a `x2` až po `z`. Pokud nyní na `z` zavoláme metodu `backward(dout)` s nějakým "příchozím" gradientem `dout` (defaultně je `None`), spustí se kompletní zpětná propagace celým výpočetním stromem až k "listům" `x1`, `a`, a `x2`.

In [15]:
z.backward()

Pro derivaci `z` podle `x1`, tj. pro gradient platí
$$\frac{dz}{dx_1} = 2(x_1 + ax_2)$$
což po dosazení $x_1 = 1$, $a = 2$ a $x_2 = 3$ vychází
$$\frac{dz}{dx_1} = 2\cdot(1 + 2 \cdot 3) = 14$$
Výsledek můžeme ověřit v PyTorchi nahlédnutím do atributu `grad` tensoru `x1`.

In [16]:
x1.grad

tensor(14.)

Ovšem pokud bychom se chtěli podívat na $dz / dy$:

In [17]:
y.grad

  y.grad


`y.grad` je `None`, protože PyTorch z důvodů šetření paměti zahazuje mezivýpočty, tj. všechny gradienty, které netvoří list stromu. Pokud bychom ho přesto chtěli vidět, lze na `y` zavolat funkci `retain_grad()` a znovu spustit zpětný průchod. K tomu však musíme znovu vytvořit celý výpočetní graf, jelikož PyTorch po zavolání backpropu defaultně vyčistí jeho buffery (toto chování lze změnit argumentem `retain_graph` metody `backward`).

In [18]:
x1 = torch.tensor(1., requires_grad=True)
a = torch.tensor(2., requires_grad=True)
x2 = torch.tensor(3., requires_grad=True)
y = x1 + a * x2
y.retain_grad()
z = y ** 2
z.backward(retain_graph=True)
y.grad

tensor(14.)

Jelikož $z = y^2$, pro gradient platí $dz / dy = 2y$, což po dosazení hodnoty $y = 7$ z dopředného průchodu znamená, že $dz / dy = 14$.

Podívejme nyní na gradienty vůči všem proměnným v grafu.

In [19]:
x1.grad, a.grad, x2.grad, y.grad, z.grad

  x1.grad, a.grad, x2.grad, y.grad, z.grad


(tensor(14.), tensor(42.), tensor(28.), tensor(14.), None)

# Akumulace gradientů: příklad $p = (x^2+1) \cdot (x^2-1)$

Poslední příklad ilustruje akumulaci gradientů v případě, kdy má jeden uzel grafu více potomků či ekvivalentně je předkem pro více uzlů.
$$s = x^2$$
$$p = s+1$$
$$m = s-1$$
$$q = p \cdot m$$
V takovém případě může vzniknout "diamantový" tvar grafu, protože na začátku je pouze jeden uzel `x`, který má dva potomky: uzly `p` a `m` (skrze `s`). Výsledek na konci, uzel `q`, pak opět své předky `p` a `m` spouje.

In [20]:
x = torch.tensor(2., requires_grad=True)

s = x ** 2; s.retain_grad()
p = s + 1; p.retain_grad()
m = s - 1; m.retain_grad()
q = p * m
s, p, m, q

(tensor(4., grad_fn=<PowBackward0>),
 tensor(5., grad_fn=<AddBackward0>),
 tensor(3., grad_fn=<SubBackward0>),
 tensor(15., grad_fn=<MulBackward0>))

Lokální derivace funkce jsou:
$$\frac{ds}{dx} = 2x$$
$$\frac{dp}{ds} = 1$$
$$\frac{dm}{ds} = 1$$
$$\frac{dq}{dp} = m$$
$$\frac{dq}{dm} = p$$

Spusťme nyní zpětnou propagaci z uzlu `q`.

In [21]:
q.backward()

Uzel `s` "uvidí" příchozí gradienty ze dvou větví, tj. z uzlů `p` a `m`, jejichž gradienty jsou:
$$\frac{dq}{dp} = m = 3$$
$$\frac{dq}{dm} = p = 5$$

In [22]:
p.grad, m.grad

(tensor(3.), tensor(5.))

V takovém případě je celková derivace dána jejich součtem:
$$\frac{dq}{ds} = \frac{dq}{dp}\frac{dp}{ds} + \frac{dq}{dm}\frac{dm}{dy} = 3 \cdot 1 + 5 \cdot 1 = 8$$

In [23]:
s.grad

tensor(8.)


Toto je důvod, proč PyTorch gradienty nepřepisuje, ale akumuluje. Při zpětné propagaci dochází k updatu `y.grad` dvakrát: jednou z větve `p` a jednou z větve `m`. Pokud by se gradient přepsal, jedna větev by "vyhrála" tím, že by updatovala jako poslední, a výsledný gradient by tak byl buď 3 nebo 5, což by ani v jednom případě nebyl správný výsledek.

In [24]:
x.grad

tensor(32.)

# `zero_grad()`

Pro zajímavost: co se stane, pokud znovu zavoláme zpětný průchod `z.backward` z minuého příkladu (proměnné stále existují ve workspace)?

In [25]:
x1.grad, a.grad, x2.grad, y.grad, z.grad

  x1.grad, a.grad, x2.grad, y.grad, z.grad


(tensor(14.), tensor(42.), tensor(28.), tensor(14.), None)

In [26]:
z.backward()

In [27]:
x1.grad, a.grad, x2.grad, y.grad, z.grad

  x1.grad, a.grad, x2.grad, y.grad, z.grad


(tensor(28.), tensor(84.), tensor(56.), tensor(28.), None)

Ačkoliv jsme s výpočetním grafem nic neprovedli, gradienty se změnily (zdvojnásobily)! Stalo se tak proto, že autograd atribut `grad` nepřepisuje, ale při každém backpropu akumuluje. Tensory `x1`, `a`, `x2`, `y` a `z` totiž nebyly znovu vytvořeny, nýbrž použity dvakrát a výsledné gradienty jsou proto *součtem* příchozích gradientů ze dvou pod-stromů. Pokud by PyTorch gradienty přepisoval, výsledek by neodpovídal pravidlům matematické analýzy.

Pokud bychom vytvořili proměnné `x1`, `a`, `x2`, `y` a `z` znovu, reference na původní objekty by přestaly existovat, z prvního výpočetního grafu by nezbylo již nic a bylo by možné je garbage collectorem vyčistit z paměti. Dokud však proměnná s "cachovaným" `grad` existuje, není možné vyčistit paměť. Z grafového pohledu by `x1`, `a`, `x2`, `y` a `z` představovaly nové uzly s ještě prázdnými gradienty, které by se tak naplnily hodnotami shodnými s prvním průchodem.

Toto chování má důležité implikace pro trénování sítí, kdy obodbným způsobem, jako v příkladu zacházíme s `x`, zacházíme s parametry sítě $W$ a $b$, např. když počítáme $s = Wx + b$. Mezi jednotlivými iteracemi SGD totiž $W$ a $b$ jako proměnné nevytváříme znovu, ale používáme stále stejné tensory. Tím vlastně dochází k opakovanému využití stejných uzlů, pokaždé ale vstupují do jiného grafu daného aktuální dávkou. Pokud před zavoláním `backprop()` tyto uzly mají nenulové gradienty `grad`, dojde k jejich akumulaci s hodnotami vypočtenými z minulé dávky. Nejenže nebudou odpovídat dopřednému průchodu na jedné dávce, ale zároveň mohou i nepříjemně narůstat. Mezi jednotlivými iteracemi je tedy nutné gradienty manuálně vynulovat, což se Pytorchi zařizuje metodou `zero_grad()` tříd `torch.nn.Module` nebo `torch.optim.Optimizer`. Nutnost manuálního volání `zero_grad()` v PyTorchi je jedním z nějčastějších zdrojů nepříjemných a špatně odhalitelných bugů.

Podrobnější vysvětlení mechanismu automatického derivování, tzv. autogradu, lze pročíst např. na webu [pytorch.org](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html).